In [1]:
import numpy as np
import scipy as sci
import scipy.constants as scicon
import pxr_qed

print("pxr_qed is compiled in {:s} precision with {:s} units".
     format(pxr_qed.PRECISION, pxr_qed.UNITS))
if(pxr_qed.HAS_OPENMP):
    print("pxr_qed has openMP support!")
else:
    print("pxr_qed does not have openMP support!")

#Warning: this notebook is conceived for SI units!!

pxr_qed is compiled in double precision with SI units
pxr_qed has openMP support!


In [2]:
m_e = scicon.electron_mass
q_e = scicon.elementary_charge
hbar = scicon.hbar
c = scicon.c
um = scicon.micron
fs = scicon.femto

E_s = m_e**2 * c**3 / (hbar * q_e)
B_s = E_s/c

In [3]:
#Chi functions

chi_how_many = 10
chi_gamma = 10

chi_Ex = (np.random.rand(chi_how_many)-0.5)*2*E_s
chi_Ey = (np.random.rand(chi_how_many)-0.5)*2*E_s
chi_Ez = (np.random.rand(chi_how_many)-0.5)*2*E_s
chi_Bx = (np.random.rand(chi_how_many)-0.5)*2*B_s
chi_By = (np.random.rand(chi_how_many)-0.5)*2*B_s
chi_Bz = (np.random.rand(chi_how_many)-0.5)*2*B_s
chi_px = (np.random.rand(chi_how_many)-0.5)*2*chi_gamma*m_e*c;
chi_py = (np.random.rand(chi_how_many)-0.5)*2*chi_gamma*m_e*c;
chi_pz = (np.random.rand(chi_how_many)-0.5)*2*chi_gamma*m_e*c;

print("chi_phot: ", pxr_qed.chi_photon(
    chi_px, chi_py, chi_pz,
    chi_Ex, chi_Ey, chi_Ez,
    chi_Bx, chi_By, chi_Bz, 1))

print("chi_ele_pos: ", pxr_qed.chi_ele_pos(
    chi_px, chi_py, chi_pz, 
    chi_Ex, chi_Ey, chi_Ez,
    chi_Bx, chi_By, chi_Bz, 1))

chi_phot:  [ 5.69987791  5.82570482 10.97872065  5.89781209 14.09537311  1.19244247
  6.74005305  6.19747655 11.90240205 16.65693906]
chi_ele_pos:  [ 5.74174727  5.80739474 11.002853    5.97281582 14.13122611  1.34429715
  6.78460622  6.22443299 11.95747209 16.69832577]


In [4]:
#Breit-Wheeler pair production

bw_dndt_params = pxr_qed.bw.dndt_lookup_table_params()
bw_dndt_params.chi_phot_min = 0.01
bw_dndt_params.chi_phot_max = 100
bw_dndt_params.chi_phot_how_many = 128
print(bw_dndt_params)

bw_dndt_lookup_table = pxr_qed.bw.dndt_lookup_table(bw_dndt_params)
print(bw_dndt_lookup_table)
bw_dndt_lookup_table.generate()
print(bw_dndt_lookup_table)

bw_dndt_lookup_table.save_as("bw_dndt.bin")
bw_dndt_lookup_table_2 = pxr_qed.bw.dndt_lookup_table()
print(bw_dndt_lookup_table_2)
bw_dndt_lookup_table_2.load_from("bw_dndt.bin");
print(bw_dndt_lookup_table_2)

bw_pair_prod_params = pxr_qed.bw.pair_prod_lookup_table_params()
bw_pair_prod_params.chi_phot_min = 0.01
bw_pair_prod_params.chi_phot_max = 100
bw_pair_prod_params.chi_phot_how_many = 64
bw_pair_prod_params.frac_how_many = 64
print(bw_pair_prod_params)

bw_pair_prod_lookup_table = pxr_qed.bw.pair_prod_lookup_table(bw_pair_prod_params)
print(bw_pair_prod_lookup_table)
bw_pair_prod_lookup_table.generate()
print(bw_pair_prod_lookup_table)

bw_pair_prod_lookup_table.save_as("bw_pairprod.bin")
bw_pair_prod_lookup_table_2 = pxr_qed.bw.pair_prod_lookup_table()
print(bw_pair_prod_lookup_table_2)
bw_pair_prod_lookup_table_2.load_from("bw_pairprod.bin");
print(bw_pair_prod_lookup_table_2)

bw_how_many = 4
bw_gamma = 10
bw_dt = 1e-2*fs

bw_Ex = (np.random.rand(bw_how_many)-0.5)*2*E_s
bw_Ey = (np.random.rand(bw_how_many)-0.5)*2*E_s
bw_Ez = (np.random.rand(bw_how_many)-0.5)*2*E_s
bw_Bx = (np.random.rand(bw_how_many)-0.5)*2*B_s
bw_By = (np.random.rand(bw_how_many)-0.5)*2*B_s
bw_Bz = (np.random.rand(bw_how_many)-0.5)*2*B_s
bw_px = (np.random.rand(bw_how_many)-0.5)*2*bw_gamma*m_e*c;
bw_py = (np.random.rand(bw_how_many)-0.5)*2*bw_gamma*m_e*c;
bw_pz = (np.random.rand(bw_how_many)-0.5)*2*bw_gamma*m_e*c
bw_rand = np.random.rand(bw_how_many)
bw_ee = np.sqrt(bw_px**2 + bw_py**2 + bw_pz**2)*c;


bw_chi =  pxr_qed.chi_photon(bw_px, bw_py, bw_pz, bw_Ex, bw_Ey, bw_Ez, bw_Bx, bw_By, bw_Bz)
print("Chi parameters: ", bw_chi, "\n")

bw_interp_dndt = bw_dndt_lookup_table.interp(bw_chi)
print("dN/dt lookup table interpolation: ", bw_interp_dndt, "\n")

bw_interp_pair_prod = bw_pair_prod_lookup_table.interp(bw_chi, bw_rand)
print("Pair production lookup table interpolation: ", bw_interp_pair_prod, "\n")

bw_opt = pxr_qed.bw.get_optical_depth(bw_rand)
print("Breit-Wheeler optical depth: ", bw_opt, "\n")

bw_dndt = pxr_qed.bw.get_dn_dt(bw_ee, bw_chi, bw_dndt_lookup_table)
print("Breit-Wheeler dNdt: ", bw_dndt, "\n")

print("Breit-Wheeler opt depth before evolution: ", bw_opt, "\n")
pxr_qed.bw.evolve_optical_depth(bw_ee, bw_chi, bw_dt, bw_opt, bw_dndt_lookup_table)
print("Breit-Wheeler opt depth after evolution: ", bw_opt, "\n")

bw_ele_px, bw_ele_py, bw_ele_pz, bw_pos_px, bw_pos_py, bw_pos_pz = pxr_qed.bw.generate_breit_wheeler_pairs(
    bw_chi, bw_px, bw_py, bw_pz, bw_rand, bw_pair_prod_lookup_table)

print("Breit-Wheeler electron px: ", bw_ele_px)
print("Breit-Wheeler electron py: ", bw_ele_py)
print("Breit-Wheeler electron pz: ", bw_ele_pz)
print("Breit-Wheeler positron px: ", bw_pos_px)
print("Breit-Wheeler positron py: ", bw_pos_py)
print("Breit-Wheeler positron pz: ", bw_pos_pz)

bw.dndt_lookup_table_params:
	chi_phot_min     : 0.01
	chi_phot_max     : 100
	chi_phot_how_many: 128
bw.dndt_lookup_table:
	is initialized? : False

bw.dndt_lookup_table:
	is initialized? : True

bw.dndt_lookup_table:
	is initialized? : False

bw.dndt_lookup_table:
	is initialized? : True

bw.pair_prod_lookup_table_params:
	chi_phot_min     : 0.01
	chi_phot_max     : 100
	chi_phot_how_many: 64
	frac_how_many    : 64
bw.pair_prod_lookup_table:
	is initialized? : False

bw.pair_prod_lookup_table:
	is initialized? : True

bw.pair_prod_lookup_table:
	is initialized? : False

bw.pair_prod_lookup_table:
	is initialized? : True

Chi parameters:  [ 4.52085808 11.13066108 14.24967391  4.23136133] 

dN/dt lookup table interpolation:  [0.0920503  0.10880774 0.10828566 0.08944404] 

Pair production lookup table interpolation:  [ 0.76526949 10.60995375  2.80380622  3.21306961] 

Breit-Wheeler optical depth:  [0.11317728 3.97356138 0.24272302 1.66933813] 

Breit-Wheeler dNdt:  [2.66096115e+17 7.730

In [5]:
# Quantum Synchrotron radiation

qs_dndt_params = pxr_qed.qs.dndt_lookup_table_params()
qs_dndt_params.chi_part_min = 0.01
qs_dndt_params.chi_part_max = 100
qs_dndt_params.chi_part_how_many = 128
print(qs_dndt_params)

qs_dndt_lookup_table = pxr_qed.qs.dndt_lookup_table(qs_dndt_params)
print(qs_dndt_lookup_table)
qs_dndt_lookup_table.generate()
print(qs_dndt_lookup_table)

qs_dndt_lookup_table.save_as("qs_dndt.bin")
qs_dndt_lookup_table_2 = pxr_qed.qs.dndt_lookup_table()
print(qs_dndt_lookup_table_2)
qs_dndt_lookup_table_2.load_from("qs_dndt.bin");
print(qs_dndt_lookup_table_2)

qs_photem_params = pxr_qed.qs.photon_emission_lookup_table_params()
qs_photem_params.chi_part_min = 0.01
qs_photem_params.chi_part_max = 100
qs_photem_params.frac_min = 1e-12
qs_photem_params.chi_part_how_many = 64
qs_photem_params.frac_how_many = 64
print(qs_photem_params)

qs_dndt_lookup_table.save_as("qs_dndt.bin")
qs_dndt_lookup_table_2 = pxr_qed.qs.dndt_lookup_table()
print(qs_dndt_lookup_table_2)
qs_dndt_lookup_table_2.load_from("qs_dndt.bin");
print(qs_dndt_lookup_table_2)

qs_photon_emission_lookup_table = pxr_qed.qs.photon_emission_lookup_table(qs_photem_params)
print(qs_photon_emission_lookup_table)
qs_photon_emission_lookup_table.generate()
print(qs_photon_emission_lookup_table)

qs_how_many = 4
qs_gamma = 10
qs_dt = 1e-2*fs

qs_Ex = (np.random.rand(qs_how_many)-0.5)*2*E_s
qs_Ey = (np.random.rand(qs_how_many)-0.5)*2*E_s
qs_Ez = (np.random.rand(qs_how_many)-0.5)*2*E_s
qs_Bx = (np.random.rand(qs_how_many)-0.5)*2*B_s
qs_By = (np.random.rand(qs_how_many)-0.5)*2*B_s
qs_Bz = (np.random.rand(qs_how_many)-0.5)*2*B_s
qs_px = (np.random.rand(qs_how_many)-0.5)*2*qs_gamma*m_e*c;
qs_py = (np.random.rand(qs_how_many)-0.5)*2*qs_gamma*m_e*c;
qs_pz = (np.random.rand(qs_how_many)-0.5)*2*qs_gamma*m_e*c;
qs_rand = np.random.rand(qs_how_many)
qs_ee = np.sqrt(1 + (qs_px**2 + qs_py**2 + qs_pz**2)/((m_e*c)**2))*m_e*c**2;


qs_chi =  pxr_qed.chi_ele_pos(qs_px, qs_py, qs_pz, qs_Ex, qs_Ey, qs_Ez, qs_Bx, qs_By, qs_Bz)
print("Chi parameters: ", qs_chi, "\n")

qs_interp_dndt = qs_dndt_lookup_table.interp(qs_chi)
print("dN/dt lookup table interpolation: ", qs_interp_dndt, "\n")

qs_interp_photon_emission = qs_photon_emission_lookup_table.interp(qs_chi, qs_rand)
print("Pair production lookup table interpolation: ", qs_interp_photon_emission, "\n")

qs_opt = pxr_qed.qs.get_optical_depth(qs_rand)
print("Quantum synchrotron optical depth: ", qs_opt, "\n")

qs_dndt = pxr_qed.qs.get_dn_dt(qs_ee, qs_chi, qs_dndt_lookup_table)
print("Quantum synchrotron dNdt: ", qs_dndt, "\n")

print("Quantum synchrotron opt depth before evolution: ", qs_opt, "\n")
pxr_qed.qs.evolve_optical_depth(qs_ee, qs_chi, qs_dt, qs_opt, qs_dndt_lookup_table)
print("Quantum synchrotron opt depth after evolution: ", qs_opt, "\n")

print("Quantum synchrotron part px (before): ", qs_px)
print("Quantum synchrotron part py (before): ", qs_py)
print("Quantum synchrotron part pz (before): ", qs_pz)

qs_phot_px, qs_phot_py, qs_phot_pz = pxr_qed.qs.generate_photon_update_momentum(
    qs_chi, qs_px, qs_py, qs_pz, qs_rand, qs_photon_emission_lookup_table)

print("Quantum synchrotron part px: ", qs_px)
print("Quantum synchrotron part py: ", qs_py)
print("Quantum synchrotron part pz: ", qs_pz)
print("Quantum synchrotron photon px: ", qs_phot_px)
print("Quantum synchrotron photon py: ", qs_phot_py)
print("Quantum synchrotron photon pz: ", qs_phot_pz)


qs.dndt_lookup_table_params:
	chi_part_min     : 0.01
	chi_part_max     : 100
	chi_part_how_many: 128
qs.dndt_lookup_table:
	is initialized? : False

qs.dndt_lookup_table:
	is initialized? : True

qs.dndt_lookup_table:
	is initialized? : False

qs.dndt_lookup_table:
	is initialized? : True

qs.photon_emission_lookup_table_params:
	chi_part_min     : 0.01
	chi_part_max     : 100
	frac_min         : 1e-12
	chi_part_how_many: 64
	frac_how_many    : 64
qs.dndt_lookup_table:
	is initialized? : False

qs.dndt_lookup_table:
	is initialized? : True

qs.pair_prod_lookup_table:
	is initialized? : False

qs.pair_prod_lookup_table:
	is initialized? : True

Chi parameters:  [14.31260562 18.67401583 19.20375739 10.46322055] 

dN/dt lookup table interpolation:  [11.87633168 14.35585401 14.64345668  9.47657392] 

Pair production lookup table interpolation:  [10.62380866  0.0798416   1.49208283  0.07908836] 

Quantum synchrotron optical depth:  [0.07761866 1.6869038  0.73837553 1.46927273] 

Quantum sy

In [6]:
#Schwinger pair production

sc_how_many = 10

sc_Ex = (np.random.rand(sc_how_many)-0.5)*2*E_s
sc_Ey = (np.random.rand(sc_how_many)-0.5)*2*E_s
sc_Ez = (np.random.rand(sc_how_many)-0.5)*2*E_s
sc_Bx = (np.random.rand(sc_how_many)-0.5)*2*B_s
sc_By = (np.random.rand(sc_how_many)-0.5)*2*B_s
sc_Bz = (np.random.rand(sc_how_many)-0.5)*2*B_s

sc_volume = um**3
sc_dt = fs

pair_production_rate = pxr_qed.sc.pair_production_rate(sc_Ex, sc_Ey, sc_Ez, sc_Bx, sc_By, sc_Bz, 1)
exp_pair_number = pxr_qed.sc.expected_pair_number(sc_Ex, sc_Ey, sc_Ez, sc_Bx, sc_By, sc_Bz, sc_volume, sc_dt, 1)

print("pair_production_rate:", pair_production_rate)

print("expected pair number:", exp_pair_number)

pair_production_rate: [4.89384424e+13 3.59995908e+16 1.23279622e-02 1.10208859e+17
 1.48899978e+16 4.91680000e-15 4.56176309e+15 2.66908513e+17
 9.06950963e+14 1.96993009e+16]
expected pair number: [6.36928574e+18 4.68530810e+21 1.60447105e+03 1.43435647e+22
 1.93791723e+21 6.39916242e-10 5.93708568e+20 3.47378565e+22
 1.18038694e+20 2.56384286e+21]
